In [3]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import scipy.stats as stats
import numpy as np
# Drop non-numeric columns which cannot be used in correlation calculation


file_path = 'dataset/cleaned_data.csv'


# Load the dataset
data_fillmean = pd.read_csv(file_path)
# Assuming 'data_fillmean' is your DataFrame

# Assuming the date column is named 'date'
data_fillmean['date'] = pd.to_datetime(data_fillmean['date'])
data_fillmean['year'] = data_fillmean['date'].dt.year  # Extract year
data_fillmean['month'] = data_fillmean['date'].dt.month  # Extract month
data_fillmean['day'] = data_fillmean['date'].dt.day  # Extract day

# Now you can drop the original 'date' column if it's no longer needed
data_fillmean.drop('date', axis=1, inplace=True)


In [ ]:
# Min-Max Scaling
min_max_scaler = MinMaxScaler()
data_minmax = min_max_scaler.fit_transform(data_fillmean.select_dtypes(include=[np.number]))
data_fillmean_minmax = pd.DataFrame(data_minmax, columns=data_fillmean.select_dtypes(include=[np.number]).columns)

# Standardization
standard_scaler = StandardScaler()
data_standardized = standard_scaler.fit_transform(data_fillmean.select_dtypes(include=[np.number]))
data_fillmean_standardized = pd.DataFrame(data_standardized, columns=data_fillmean.select_dtypes(include=[np.number])..columns)

# Display the first few rows to confirm the transformation
print(data_fillmean_minmax.head())
print(data_fillmean_standardized.head())